### import package

In [5]:
import re

In [6]:
def read_file(path, encoding="utf-8-sig"):
    with open(path, encoding=encoding) as fr:
        return fr.readlines()

In [7]:
PHI_KEYS = [
    "PATIENT",
    "DOCTOR",
    "USERNAME",
    "PROFESSION",
    "ROOM",
    "DEPARTMENT",
    "HOSPITAL",
    "ORGANIZATION",
    "STREET",
    "CITY",
    "STATE",
    "COUNTRY",
    "ZIP",
    "LOCATION-OTHER",
    "AGE",
    "DATE",
    "TIME",
    "DURATION",
    "SET",
    "PHONE",
    "FAX",
    "EMAIL",
    "URL",
    "IPADDR",
    "SSN",
    "MEDICALRECORD",
    "HEALTHPLAN",
    "ACCOUNT",
    "LICENSE",
    "VECHICLE",
    "DEVICE",
    "BIOID",
    "IDNUM",
    "OTHER",
]

NORMALIZED_PHI_KEYS = ["DATE", "TIME", "DURATION", "SET"]

In [9]:
filename = "2023-11-27_08-47-30_answer"
filename = "esun_answer"
filename = "2023-12-02_13-30-31_answer"


valid_data = dict()
for valid in read_file(f"../out/opendid_test.tsv"):
    if valid == "\n":
        continue
    [fid, start, content] = valid.split("\t")
    if fid in valid_data:
        valid_data[fid].append((start, content))
    else:
        valid_data[fid] = [(start, content)]


with open(f"../out/{filename}_post.txt", "w", encoding="utf-8") as fw:
    for answer in read_file(f"../out/{filename}.txt"):
        ### == Data sample ===========================================
        ### fid     	phi     start   end     value       normalized
        ### file8786	DOCTOR	2975	2984	V Strimel
        ### file8786	DATE	4462	4471	12/6/2067	2067-06-12
        ### ==========================================================
        row = answer.rstrip().split("\t")
        if len(row) < 5:
            continue
        fid, phi, start, end, value, *normalized = row
        # if fid != 'file9392':
        #     continue
        if len(normalized) > 0 and "\\n" in normalized[0]:
            normalized = [normalized[0].split("\\n", 1)[0]]

        if "\\n" not in value:
            if phi in PHI_KEYS:
                if phi in NORMALIZED_PHI_KEYS and len(normalized) == 0:
                    # print(phi, value)
                    continue
                fw.write("\t".join([fid, phi, start, end, value, *normalized]))
                fw.write("\n")
            # print(answer)
            continue

        phi_info = phi + ": " + value
        # print(phi_info, start, end, normalized)
        phi_maps = list(dict.fromkeys(phi_info.split("\\n")))
        for phi_item in phi_maps:
            # print(phi_item)
            if ": " not in phi_item:
                continue
            [phi, value] = phi_item.split(": ", 1)
            if phi not in PHI_KEYS:
                continue
            for sindex, content in valid_data[fid]:
                if int(sindex) == int(start) + 1:
                    for found in re.finditer(re.escape(value), content):
                        new_start = int(sindex) + found.start()
                        new_end = int(sindex) + found.end()

                        if phi in NORMALIZED_PHI_KEYS and len(normalized) == 0:
                            # print(phi, value)
                            continue

                        fw.write(
                            "\t".join(
                                [
                                    fid,
                                    phi,
                                    str(new_start),
                                    str(new_end),
                                    value,
                                    *(normalized if phi in NORMALIZED_PHI_KEYS else []),
                                ]
                            )
                        )
                        fw.write("\n")
                        # print('\t'.join([fid, phi, str(new_start), str(new_end), value, *(normalized if phi in NORMALIZED_PHI_KEYS else [])]))
                    break